In [41]:
import pandas as pd
import numpy as  np
'''
Pandas是一个开源的Python数据分析库。Pandas把结构化数据分为了三类：
Series，1维序列，可视作为没有column名的、只有一个column的DataFrame；
DataFrame，同Spark SQL中的DataFrame一样，其概念来自于R语言，为多column并schema化的2维结构化数据，可视作为Series的容器（container）；
Panel，为3维的结构化数据，可视作为DataFrame的容器；
DataFrame较为常见，因此本文主要讨论内容将为DataFrame。DataFrame的生成可通过读取纯文本、Json等数据来生成，亦可以通过Python对象来生成：
'''
df = pd.DataFrame({'total_bill': [16.99, 10.34, 23.68, 23.68, 24.59],
                   'tip': [1.01, 1.66, 3.50, 3.31, 3.61],
                   'sex': ['Female', 'Male', 'Male', 'Male', 'Female']})

In [3]:
df.values

array([['Female', 1.01, 16.99],
       ['Male', 1.66, 10.34],
       ['Male', 3.5, 23.68],
       ['Male', 3.31, 23.68],
       ['Female', 3.61, 24.59]], dtype=object)

In [11]:
df

,sex,tip,total_bill
0,Female,1.01,16.99
1,Male,1.66,10.34
2,Male,3.50,23.68
3,Male,3.31,23.68
4,Female,3.61,24.59


In [5]:
df.index

RangeIndex(start=0, stop=5, step=1)

In [6]:
df.columns

Index(['sex', 'tip', 'total_bill'], dtype='object')

In [7]:
df.shape

(5, 3)

In [8]:
print(df.loc[1:3,['total_bill','tip']])

   total_bill   tip
1       10.34  1.66
2       23.68  3.50
3       23.68  3.31


In [9]:
print(df.loc[1:3,'tip':'total_bill'])

    tip  total_bill
1  1.66       10.34
2  3.50       23.68
3  3.31       23.68


In [10]:
print(df.iloc[1:3,[1,2]])

    tip  total_bill
1  1.66       10.34
2  3.50       23.68


In [12]:
print(df.at[3,'tip'])

3.31


In [13]:
print(df[1:3])

    sex   tip  total_bill
1  Male  1.66       10.34
2  Male  3.50       23.68


In [14]:
print(df[['total_bill','tip']])

   total_bill   tip
0       16.99  1.01
1       10.34  1.66
2       23.68  3.50
3       23.68  3.31
4       24.59  3.61


In [15]:
print(df[df['sex']=='Female'])

      sex   tip  total_bill
0  Female  1.01       16.99
4  Female  3.61       24.59


In [16]:
#Pandas实现where filter，较为常用的办法为df[df[colunm] boolean expr]，比如：
print(df[df['total_bill']>20])

      sex   tip  total_bill
2    Male  3.50       23.68
3    Male  3.31       23.68
4  Female  3.61       24.59


In [17]:
print(df.query('total_bill>20'))

      sex   tip  total_bill
2    Male  3.50       23.68
3    Male  3.31       23.68
4  Female  3.61       24.59


In [18]:
#在where子句中常常会搭配and, or, in, not关键词，Pandas中也有对应的实现：
#and
print(df[(df['sex']=='Female')&(df['total_bill']>20)])

      sex   tip  total_bill
4  Female  3.61       24.59


In [19]:
#or
print(df[(df['sex']=='Female') | (df['total_bill']>20)])

      sex   tip  total_bill
0  Female  1.01       16.99
2    Male  3.50       23.68
3    Male  3.31       23.68
4  Female  3.61       24.59


In [20]:
#in
print(df[df['total_bill'].isin([21.01, 23.68, 24.59])])

      sex   tip  total_bill
2    Male  3.50       23.68
3    Male  3.31       23.68
4  Female  3.61       24.59


In [26]:
#string function
print(df[(df['sex'].str.contains('Male\d+$'))])

Empty DataFrame
Columns: [sex, tip, total_bill]
Index: []


In [32]:
#对where条件筛选后只有一行的dataframe取其中某一列的值，其两种实现方式如下：
total = df.loc[df['tip']==1.66,['sex','tip','total_bill']].values[0]

In [33]:
print(total)

['Male' 1.66 10.34]


In [37]:
#只能取某一列的值，上面的方法可以取多列的值
total = df.get_value(df.loc[df['tip'] == 1.66].index.values[0], 'total_bill')

In [38]:
total

10.34

In [39]:
#drop_duplicates根据某列对dataframe进行去重：
'''
包含参数：

subset，为选定的列做distinct，默认为所有列；
keep，值选项{'first', 'last', False}，保留重复元素中的第一个、最后一个，或全部删除；
inplace ，默认为False，返回一个新的dataframe；若为True，则返回去重后的原dataframe
'''
df.drop_duplicates(subset=['sex'],keep='first',inplace=True)

In [40]:
df

,sex,tip,total_bill
0,Female,1.01,16.99
1,Male,1.66,10.34


In [42]:
df

,sex,tip,total_bill
0,Female,1.01,16.99
1,Male,1.66,10.34
2,Male,3.50,23.68
3,Male,3.31,23.68
4,Female,3.61,24.59


In [43]:
'''
group一般会配合合计函数（Aggregate functions）使用，
比如：count、avg等。Pandas对合计函数的支持有限，有count和size函数实现SQL的count：
'''
print(df.groupby('sex').size())

sex
Female    2
Male      3
dtype: int64


In [44]:
print(df.groupby('sex').count())

        tip  total_bill
sex                    
Female    2           2
Male      3           3


In [45]:
print(df.groupby('sex')['tip'].count())

sex
Female    2
Male      3
Name: tip, dtype: int64


In [46]:
print(df.groupby('sex')['tip'].size())

sex
Female    2
Male      3
Name: tip, dtype: int64


In [47]:
#对于多合计函数，
'''
select sex, max(tip), sum(total_bill) as total
from tips_tb
group by sex;
'''
#实现在agg()中指定dict：
print(df.groupby('sex').agg({'tip':np.max,'total_bill':np.sum}))


         tip  total_bill
sex                     
Female  3.61       41.58
Male    3.50       57.70


In [48]:
# count(distinct **)
print(df.groupby('tip').agg({'sex':pd.Series.nunique}))


      sex
tip      
1.01    1
1.66    1
3.31    1
3.50    1
3.61    1


In [49]:
#SQL中使用as修改列的别名，Pandas也支持这种修改：
df1=df.rename(columns={'total_bill': 'total', 'tip': 'pit', 'sex': 'xes'}, inplace=False)
print(df1)

      xes   pit  total
0  Female  1.01  16.99
1    Male  1.66  10.34
2    Male  3.50  23.68
3    Male  3.31  23.68
4  Female  3.61  24.59


In [51]:
#Pandas中join的实现也有两种：
#df3 = pd.merge(df1,df2,how='inner',on='门店Id')

In [53]:
#Pandas中支持多列order，并可以调整不同列的升序/降序，有更高的排序自由度：
print(df.sort_values(['total_bill','tip'],ascending=[False,True]))

      sex   tip  total_bill
4  Female  3.61       24.59
3    Male  3.31       23.68
2    Male  3.50       23.68
0  Female  1.01       16.99
1    Male  1.66       10.34


In [66]:
#对于全局的top：
print(df.nlargest(3,columns=['total_bill']))

      sex   tip  total_bill
4  Female  3.61       24.59
2    Male  3.50       23.68
3    Male  3.31       23.68


In [11]:
import pytesseract
from PIL import Image

In [13]:
#C:\Program Files (x86)\Tesseract-OCR\tesseract.exe
#pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files (x86)\Tesseract-OCR\tesseract.exe'
print(pytesseract.image_to_string(Image.open('cc.jpg'),lang='chi_sim'))

TesseractError: (1, 'Error opening data file \\Program Files (x86)\\Tesseract-OCR\\tessdata/chi_sim.traineddata')